Example usage of the provided "cython_peg.py" parsing file.  This code is not mature/complex enough to be a python module.  
The parsing will probably need to be adjusted for specific project needs.  The pyparsing syntax is dense but powerful.  
It will take some document reading to learn the pyparsing syntax and make edits.

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
from cython_peg import cython2stub
from pathlib import Path

# basic import template
# easier than transfering cython imports and modifying used/unused imports
# 90% of cython files use the basic types
# memmory views colons are transfered as the type COLON
import_template = "\n".join([
"import numpy as np",
"from enum import Enum",
"from typing import Optional, Tuple, Dict, List, Generator, Union, DefaultDict",
"\n# type convertions from cython to python",
"float64 = float32 = double = long = longlong = float",
"uint64 = uint32 = uint16 = uint8 = short = int",
"COLON = str",
"\n"])

examples = Path(r"../examples/").glob("*.pyx")
for example in examples:
    
    # generate stub file
    stub_file = cython2stub(example)
    
    # g

    
    # save the type file
    with open(example.with_suffix(".pyi"), mode='w') as f:
        f.write(import_template+stub_file)
